### Dependencies

In [1]:
# !pip install pypdf langchain langchain_core langchain_community langchain_huggingface langchain_groq sentence_transformers transformers langchain_google_genai faiss-cpu langchain_openai grandalf pymupdf

### Loading PDF

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("GSTsmartGuide.pdf")

In [4]:
docs = loader.load()

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-02-27T11:53:51+05:30', 'source': 'GSTsmartGuide.pdf', 'file_path': 'GSTsmartGuide.pdf', 'total_pages': 1321, 'format': 'PDF 1.6', 'title': 'CHAPTER 1', 'author': 'Abha', 'subject': '', 'keywords': '', 'moddate': '2024-02-27T13:03:27+05:30', 'trapped': '', 'modDate': "D:20240227130327+05'30'", 'creationDate': "D:20240227115351+05'30'", 'page': 0}, page_content='')

In [5]:
len(docs)

1321

In [6]:
docs[10]

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-02-27T11:53:51+05:30', 'source': 'GSTsmartGuide.pdf', 'file_path': 'GSTsmartGuide.pdf', 'total_pages': 1321, 'format': 'PDF 1.6', 'title': 'CHAPTER 1', 'author': 'Abha', 'subject': '', 'keywords': '', 'moddate': '2024-02-27T13:03:27+05:30', 'trapped': '', 'modDate': "D:20240227130327+05'30'", 'creationDate': "D:20240227115351+05'30'", 'page': 10}, page_content='10 \nContents \n \nChapter \nTopic Subject wise \nSection of \nCGST \nAct \nRule of  \nCGST \nRules \nPage \nNo. \nChapter 21 \nAdvance Ruling \n96 to 101 \n103 to 107 \n543 \nChapter 22 \nAppeals and Revision \n107 to 115 108 to 115 \n562 \nChapter 23 \nOffence and Penalties \n122 to 138 \n162 \n593 \nChapter 24 \nAnti-Profiteering Measures \n171 \n122 to 137 \n620 \nChapter 25 \nJob Work under GST \n143 \n \n632 \nChapter 26 \nElectronic Way Bill \n \n138 to 138D 651 \nChapter 27 \nAuthorised Representatives \n116 \n

## Hybrid approaches

In [8]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [7]:
import nltk
import tiktoken
from sentence_transformers import SentenceTransformer, util
import numpy as np

nltk.download('punkt')

# Initialize tools
model = SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    return len(tokenizer.encode(text))

def split_into_paragraphs(text):
    return [p.strip() for p in text.split("\n\n") if p.strip()]

def hybrid_chunking_semantic_fixed_limit(docs, max_tokens_per_chunk=512):
    """
    Splits documents into semantically meaningful chunks without exceeding the token limit.

    Args:
        docs (list): List of documents with .page_content attribute.
        max_tokens_per_chunk (int): Maximum number of tokens allowed in each chunk.

    Returns:
        list: List of text chunks.
    """

    # Step 1: Combine all text from the docs
    full_text = "\n\n".join(doc.page_content for doc in docs if doc.page_content.strip())

    # Step 2: Paragraph-level split
    paragraphs = split_into_paragraphs(full_text)

    # Step 3: Greedy grouping under token limit
    chunks, current_chunk, current_tokens = [], [], 0

    for para in paragraphs:
        para_tokens = count_tokens(para)

        # If a single paragraph is too big, split it sentence-wise
        if para_tokens > max_tokens_per_chunk:
            sentences = nltk.sent_tokenize(para)
            for sentence in sentences:
                sentence_tokens = count_tokens(sentence)
                if current_tokens + sentence_tokens <= max_tokens_per_chunk:
                    current_chunk.append(sentence)
                    current_tokens += sentence_tokens
                else:
                    chunks.append(" ".join(current_chunk))
                    current_chunk = [sentence]
                    current_tokens = sentence_tokens
        else:
            if current_tokens + para_tokens <= max_tokens_per_chunk:
                current_chunk.append(para)
                current_tokens += para_tokens
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [para]
                current_tokens = para_tokens

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
hybrid_chunks = hybrid_chunking_semantic_fixed_limit(docs, max_tokens_per_chunk=200)

In [11]:
hybrid_chunks[100]

'1 \nGST — Concept & Status  \n19 \n \n25.28 Real Estate: \n25.29 The GST Council in its 33rd & 34th meetings held on 24-2-2019 &  \n19-3-2019 respectively have made following decisions with respect to the real \nestate sector: \n1.25.30 GST shall be levied at effective rate of 5% on residential properties \noutside affordable segment and 1% on affordable housing properties. 25.31 Definition of affordable housing:  \nA residential house/flat of carpet area of up to 90 sqm in non-metropolitan \ncities/towns and 60 sqm in metropolitan cities having value up to `45 lacs (both \nfor metropolitan and non-metropolitan cities).'